In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify the API key is loaded
if os.getenv("TINKER_API_KEY"):
    print("✓ API key loaded successfully")
else:
    print("✗ Warning: TINKER_API_KEY not found in .env file")

✓ API key loaded successfully


In [4]:
import tinker
service_client = tinker.ServiceClient()
print("Available models:")
for item in service_client.get_server_capabilities().supported_models:
    print("- " + item.model_name)

Available models:
- deepseek-ai/DeepSeek-V3.1
- deepseek-ai/DeepSeek-V3.1-Base
- meta-llama/Llama-3.1-70B
- meta-llama/Llama-3.1-8B
- meta-llama/Llama-3.1-8B-Instruct
- meta-llama/Llama-3.2-1B
- meta-llama/Llama-3.2-3B
- meta-llama/Llama-3.3-70B-Instruct
- Qwen/Qwen3-235B-A22B-Instruct-2507
- Qwen/Qwen3-30B-A3B
- Qwen/Qwen3-30B-A3B-Base
- Qwen/Qwen3-30B-A3B-Instruct-2507
- Qwen/Qwen3-32B
- Qwen/Qwen3-4B-Instruct-2507
- Qwen/Qwen3-8B
- Qwen/Qwen3-8B-Base
- openai/gpt-oss-120b
- openai/gpt-oss-20b


In [5]:
base_model = "Qwen/Qwen3-30B-A3B-Base"
training_client = service_client.create_lora_training_client(
    base_model=base_model
)

In [7]:
print(training_client)

In [9]:
# Create some training examples
examples = [
    {
        "input": "banana split",
        "output": "anana-bay plit-say"
    },
    {
        "input": "quantum physics",
        "output": "uantum-qay ysics-phay"
    },
    {
        "input": "donut shop",
        "output": "onut-day op-shay"
    },
    {
        "input": "pickle jar",
        "output": "ickle-pay ar-jay"
    },
    {
        "input": "space exploration",
        "output": "ace-spay exploration-way"
    },
    {
        "input": "rubber duck",
        "output": "ubber-ray uck-day"
    },
    {
        "input": "coding wizard",
        "output": "oding-cay izard-way"
    },
]
 
# Convert examples into the format expected by the training client
from tinker import types
 
# Get the tokenizer from the training client
tokenizer = training_client.get_tokenizer()
 
def process_example(example: dict, tokenizer) -> types.Datum:
    # Format the input with Input/Output template
    # For most real use cases, you'll want to use a renderer / chat template,
    # (see later docs) but here, we'll keep it simple.
    prompt = f"English: {example['input']}\nPig Latin:"
 
    prompt_tokens = tokenizer.encode(prompt, add_special_tokens=True)
    prompt_weights = [0] * len(prompt_tokens)
    # Add a space before the output string, and finish with double newline
    completion_tokens = tokenizer.encode(f" {example['output']}\n\n", add_special_tokens=False)
    completion_weights = [1] * len(completion_tokens)
 
    tokens = prompt_tokens + completion_tokens
    weights = prompt_weights + completion_weights
 
    input_tokens = tokens[:-1]
    target_tokens = tokens[1:] # We're predicting the next token, so targets need to be shifted.
    weights = weights[1:]
 
    # A datum is a single training example for the loss function.
    # It has model_input, which is the input sequence that'll be passed into the LLM,
    # loss_fn_inputs, which is a dictionary of extra inputs used by the loss function.
    return types.Datum(
        model_input=types.ModelInput.from_ints(tokens=input_tokens),
        loss_fn_inputs=dict(weights=weights, target_tokens=target_tokens)
    )
 
processed_examples = [process_example(ex, tokenizer) for ex in examples]
 
# Visualize the first example for debugging purposes
datum0 = processed_examples[0]
print(f"{'Input':<20} {'Target':<20} {'Weight':<10}")
print("-" * 50)
for i, (inp, tgt, wgt) in enumerate(zip(datum0.model_input.to_ints(), datum0.loss_fn_inputs['target_tokens'].tolist(), datum0.loss_fn_inputs['weights'].tolist())):
    print(f"{repr(tokenizer.decode([inp])):<20} {repr(tokenizer.decode([tgt])):<20} {wgt:<10}")

/opt/homebrew/Caskroom/miniforge/base/envs/tinker-notebook/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception ignored in: <function InternalClientHolder.__del__ at 0x13c115260>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/tinker-notebook/lib/python3.11/site-packages/tinker/lib/internal_client_holder.py", line 231, in __del__
    self.close()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/tinker-notebook/lib/python3.11/site-packages/tinker/lib/internal_client_holder.py", line 226, in close
    self.run_coroutine_threadsafe(self._async_cleanup()).result()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/tinker-notebook/lib/python3.11/site-packages/tinker/lib/public_interfaces/api_future.py", line 33, in result
    return self._future.result(timeout)
  

Input                Target               Weight    
--------------------------------------------------
'English'            ':'                  0.0       
':'                  ' banana'            0.0       
' banana'            ' split'             0.0       
' split'             '\n'                 0.0       
'\n'                 'P'                  0.0       
'P'                  'ig'                 0.0       
'ig'                 ' Latin'             0.0       
' Latin'             ':'                  0.0       
':'                  ' an'                1.0       
' an'                'ana'                1.0       
'ana'                '-b'                 1.0       
'-b'                 'ay'                 1.0       
'ay'                 ' pl'                1.0       
' pl'                'it'                 1.0       
'it'                 '-s'                 1.0       
'-s'                 'ay'                 1.0       
'ay'                 '\n\n'               1.0   

In [10]:
import numpy as np
for _ in range(6):
    fwdbwd_future = training_client.forward_backward(processed_examples, "cross_entropy")
    optim_future = training_client.optim_step(types.AdamParams(learning_rate=1e-4))
 
    # Wait for the results
    fwdbwd_result = fwdbwd_future.result()
    optim_result = optim_future.result()
 
    # fwdbwd_result contains the logprobs of all the tokens we put in. Now we can compute the weighted
    # average log loss per token.
    logprobs = np.concatenate([output['logprobs'].tolist() for output in fwdbwd_result.loss_fn_outputs])
    weights = np.concatenate([example.loss_fn_inputs['weights'].tolist() for example in processed_examples])
    print(f"Loss per token: {-np.dot(logprobs, weights) / weights.sum():.4f}")

Loss per token: 1.0631
Loss per token: 0.8720
Loss per token: 0.5823
Loss per token: 0.3476
Loss per token: 0.1894
Loss per token: 0.0979


In [11]:
# First, create a sampling client. We need to transfer weights
sampling_client = training_client.save_weights_and_get_sampling_client(name='pig-latin-model')
 
# Now, we can sample from the model.
prompt=types.ModelInput.from_ints(tokenizer.encode("English: coffee break\nPig Latin:"))
params = types.SamplingParams(max_tokens=20, temperature=0.0, stop=["\n"]) # Greedy sampling
future = sampling_client.sample(prompt=prompt, sampling_params=params, num_samples=8)
result = future.result()
print("Responses:")
for i, seq in enumerate(result.sequences):
    print(f"{i}: {repr(tokenizer.decode(seq.tokens))}")

Responses:
0: ' affe-coy eak-bay\n\n'
1: ' offy-pay eak-break-pay\n\n'
2: ' offy-cay eak-bray\n'
3: ' offy-ciay eak-brey\n\n'
4: ' affee-ca-y ockey-bay\n'
5: ' offy-pay eak-bay\n\n'
6: ' offer-cay eak-bray\n\n'
7: ' offey -ay eak -bray\n\n'


In [12]:
resume_path = training_client.save_state(name="play_101").result().path

In [13]:
print(resume_path)

tinker://5b938079-2f3a-53c0-ab4c-6930dd2a5887:train:0/weights/play_101


In [15]:
print("tinker://5b938079-2f3a-53c0-ab4c-6930dd2a5887:train:0/sampler_weights/ephemeral_13")

tinker://5b938079-2f3a-53c0-ab4c-6930dd2a5887:train:0/sampler_weights/ephemeral_13
